In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect, text
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker, declarative_base
from werkzeug.security import generate_password_hash

In [2]:
# Формируем строку подключения
connection_string = open('../interface_db.txt', 'r').read()
# Создаем движок подключения
engine = create_engine(connection_string)
# Проверяем подключение
try:
    with engine.connect() as connection:
        print("Подключение к базе данных успешно!")
except Exception as e:
    print(f"Ошибка подключения к базе данных: {e}")

Подключение к базе данных успешно!


In [11]:
inspector = inspect(engine)
tables = inspector.get_table_names()
display(pd.DataFrame(tables, columns=['Table Name']))

,Table Name
0,interface_user


# Пользователь

## Создание таблицы user

In [8]:
Base = declarative_base()

In [9]:
class InterfaceUser(Base):
    __tablename__ = 'interface_user'

    id = Column(Integer, primary_key=True)
    first_name = Column(String, nullable=False)  # Имя
    last_name = Column(String, nullable=False)   # Фамилия
    username = Column(String, unique=True, nullable=False)  # Логин
    password = Column(String, nullable=False)  # Пароль

In [10]:
Base.metadata.create_all(engine)

In [6]:
#Base.metadata.drop_all(engine, tables=[InterfaceUser.__table__])  # удаление тыблицы

## Добавление пользователей

In [12]:
Session = sessionmaker(bind=engine)
session = Session()

In [14]:
def add_user(first_name, last_name, username, password):
    user = InterfaceUser(
        first_name=first_name,
        last_name=last_name,
        username=username,
        password=generate_password_hash(password)
    )
    session.add(user)
    session.commit()
    print(f"Пользователь {username} добавлен!")

## Получение всех пользователей

In [16]:
query = text("SELECT * FROM interface_user")
with engine.connect() as connection:
    result = connection.execute(query)
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

# Выведите DataFrame
print(df.head())

   id first_name  last_name   username  \
0   1    Дмитрий  Кондрашов  dmitrykon   

                                            password  
0  scrypt:32768:8:1$vvIpmHn0o6OCKPpC$b22cf5e24227...  
